In [ ]:
import time

# from IPython.display import HTML

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcdefaults()
import torch
# from tqdm import tqdm

from threedeequadsim import quadsim, controller, trajectory, quadrotoranimation, experiments, utils

import importlib
importlib.reload(controller)
importlib.reload(quadsim)
importlib.reload(trajectory)
importlib.reload(quadrotoranimation)

In [ ]:
folder = './plots/' + time.strftime('%Y-%m-%d') + '/'
if not os.path.isdir(folder):
    os.makedirs(folder)
    print('Created data folder ' + folder)

In [ ]:
# def savefig(plottag):
#     plt.savefig(folder + nametag + '_' + plottag, bbox_inches='tight')
def savefig(plottag):
    pass
xyz = ['x', 'y', 'z']

In [ ]:
eta_a = 0.005

eta_A_convex = 0.0005
eta_A_biconvex = 0.0002
eta_A_deep = 0.05

dim_a = 50
dim_A = 100
layer_sizes = (75, 150)

feature_freq = 0.25

c = controller.MetaAdaptBaseline(eta_a_base=eta_a, dim_a=dim_a, feature_freq=feature_freq)

In [ ]:
np.random.seed(117) # Does this work when np is loaded separately in packages?
# c = controller.MetaAdaptBiconvex(dim_a=100, dim_A=100, eta_a_base=0.005, eta_A_base=0.002)
# c= controller.MetaAdaptBiconvex(dim_a = 50, dim_A=50, eta_A_base=0.001, eta_a_base=0.001)
# c = controller.MetaAdaptConvex()
# c = controller.MetaAdaptDeep(eta_A_base=0.01, eta_a_base=0.01)
# c = controller.MetaAdaptBaseline()
# c = controller.Baseline()

Vwind = np.array((3., 0., 0.))
Vwind_cov = 1.0

q_kwargs = {
    'Vwind' : 0.,                       # mean wind speed
    'wind_model': 'iid-uniform',                # {'iid', 'random-walk'}
    'Vwind_cov' : 3.0,                  # how quickly the wind changes
    # 'wind_constraint' : 'hard',         # 'hard' wind constraint limits wind speed to be within Vwind_gust of the mean speed
    'Vwind_gust' : 5.0,                # for hard wind constrant, wind speed is in the range Vwind +/- Vwind_gust
    'wind_update_period' : 2.0,         # seconds between wind speed changes
    't_stop' : 30.,
}
q = quadsim.QuadrotorWithSideForce(**q_kwargs, )
# q = quadsim.QuadrotorWithSideForce(Vwind=Vwind, Vwind_cov=Vwind_cov, wind_constraint='hard', Vwind_gust=1.0)# Quadrotor
# q = quadsim.Quadrotor() # Quadrotor
# q.params['t_stop'] = 20.


T = 20.
# t = trajectory.fig8(T=T, dir1=(0., 0., 2.*np.sqrt(2.)), dir2=(1./np.sqrt(2.), 1./np.sqrt(2.), 0.))
# t = trajectory.fig8(T=T, dir1=(0.75, 0., 0.), dir2=(0, 0., 0.4))
t = trajectory.hover(np.array((0., 0., 0.)))
# t = trajectory.random_setpoint_generator(seed=116, T=T)
# t = trajectory.SmoothSetpoint(time_scale=T, vertices=((0., 0., 0.), (1., 0., 1.), (1., 0., 1.), (-1., 0., 1.), (-1., 0., 1.), 
    # (1., 0., -1.), (1., 0., -1.), (-1., 0., -1.), (-1., 0., -1.)))
# t = trajectory.HourGlass()

tag = '' #('_W-%0.2f' % W).replace('.', 'd')
nametag = c._name + '_' + t._name + tag + '_wind=' + str(int(Vwind[0]))
nametag_short = nametag.split('_')[-1]
print(folder)
print(nametag)
time.sleep(0.1)

# %prun -D profile.prof data = q.run(trajectory=t, controller=c)
data = q.run(trajectory=t, controller=c, seed=None)
log = utils.dotdict(data)

In [ ]:
# %debug

In [ ]:
# a = quadrotoranimation.animate(data, fig=plt.figure(figsize=(10,10)))
# # HTML(a.to_jshtml())
# a.save('plot-gif_' + nametag + '.gif', writer='imagemagick', fps=30)

In [ ]:

# importlib.reload(quadrotoranimation)
# a = quadrotoranimation.draw_frames(data, fig=plt.figure(figsize=(10,10)))

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
plt.quiver([0], [0], [0], [1], [1], [1], normalize=True)


In [ ]:
fig = plt.figure(figsize=(10,5))
fig.add_subplot(121, projection='3d')
plt.plot(log['X'][:,0],log['X'][:,1], log.X[:,2])
plt.plot(log['pd'][:,0], log['pd'][:,1], log.pd[:,2])
ax = plt.gca()
bound = 3
ax.set_xlim(-bound,bound)
ax.set_ylim(-bound,bound)
ax.set_zlim(-bound,bound)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
fig.add_subplot(122)
plt.plot(log['X'][:,0],log['X'][:,2])
plt.plot(log['pd'][:,0], log['pd'][:,2])
plt.axis((-bound,bound,-bound,bound))
plt.xlabel('x')
plt.ylabel('z')
# tstart = 15.
# istart = int(100*tstart)
starttime = 0. # 0 * T
istart = int(100 * starttime)
rmse = np.sqrt(np.mean(np.sum((log['X'][istart:, 0:3] - log['pd'][istart:])**2,1)))
rmse_all = np.sqrt(np.mean(np.sum((log['X'][:, 0:3] - log['pd'])**2,1)))
maxerror = np.max(np.sqrt(np.sum((log['X'][istart:, 0:3] - log['pd'][istart:])**2,1)))
meanerror = np.mean(np.sqrt(np.sum((log['X'][istart:, 0:3] - log['pd'][istart:])**2,1)))
print('rmse = ', rmse)
print('rmse_all = ', rmse_all)
plt.title('after %.1fs, \n' % log.t[istart] + 'rmspe=' + '%.3fm' % rmse + '\nmean error=%.3fm, \nmax error=%.3f\n' % (meanerror, maxerror) + nametag)
# plt.savefig('tracking-error_' + nametag + '.png')
# plt.savefig('plots/plot-3D-' + nametag + '.jpg')
savefig('3D')

plt.figure(figsize=(15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.plot(log['t'], log['X'][:,i], label=xyz[i])
    plt.plot(log['t'], log['pd'][:,i], label=xyz[i] + '_d')
    plt.plot(log['t'], log.Vwind[:,i], label='Vwind_' + xyz[i])
    plt.legend()
    plt.xlabel('t')
    plt.ylabel(xyz[i])

plt.show()
# plt.savefig('plots/plot-xyz-' + nametag + '.jpg')
savefig('xyzwind')

In [ ]:
plt.figure(figsize=(15,5))
istart = int(100 * starttime)
for i, label in enumerate(('x', 'y', 'z')):
    plt.subplot(1,3,i+1)
    idxmin = 0
    idxtmax = 10000
    plt.plot(log.t[idxmin:idxtmax], log.Fs[idxmin:idxtmax,i])
    plt.plot(log.t[idxmin:idxtmax], log.y[idxmin:idxtmax,i])
    plt.plot(log.t[idxmin:idxtmax], log.f_hat[idxmin:idxtmax,0,i])
    plt.legend(('Fs', 'y', 'fhat'))
    for t in log['t'][log['meta_adapt_trigger']]:
        plt.axvline(t, ls=':', color='k')
    ymin, ymax = plt.ylim()
    plt.plot([log.t[istart], log.t[istart]], [ymin, ymax], 'k:')
    meanerror = np.mean(np.abs(log.Fs[istart:,i] - log.f_hat[istart:,0,i]))
    maxerror = np.max(np.abs(log.Fs[istart:,i] - log.f_hat[istart:,0,i]))
    rmse = np.sqrt(np.mean((log.Fs[istart:, i] - log.f_hat[istart:, 0, i])**2))
    plt.title(label + ', \nmeanerror = %.2fN' % meanerror + '\nrmse=%.2fN' % rmse + '\nmax error=%.2fN' % maxerror + '\n' + nametag)
    
plt.ylim(bottom=-5, top=5)
ax = plt.gca()
# ax.set_ylim(-5,5)    
plt.annotate('*Fs and y differ due to imu noise', xy=(0.05, 0.02), xycoords='figure fraction')
# plt.savefig('plots/plot-f-' + nametag + '.png', bbox_inches='tight')
plt.show()
savefig('fhat')
# plt.savefig('plot-f-err-due-to-motor-delay-2000Hz-learned.png')

In [ ]:
# Plot the components of F_r
plt.figure(figsize=(15,5))
idx_start = 0
idx_end = 10000
for i in range(3):
    plt.subplot(1,4,i+1)
    # plt.plot(log.t[idx_start:idx_end], log.p_error[idx_start:idx_end,i], label='p err')
    # plt.plot(log.t[idx_start:idx_end], log.v_error[idx_start:idx_end,i], label='v err')
    plt.plot(log.t[idx_start:idx_end], log.p_term[idx_start:idx_end,i], label='p term')
    plt.plot(log.t[idx_start:idx_end], log.v_term[idx_start:idx_end,i], label='v term')
    try:
        plt.plot(log.t[idx_start:idx_end], log.f_hat[idx_start:idx_end,0,i], label='f_hat term')
    except:
        print("no f-hat")
    plt.plot(log.t[idx_start:idx_end], log.i_term[idx_start:idx_end,i], label='i term')
    plt.plot(log.t[idx_start:idx_end], log.ad_term[idx_start:idx_end,i], label='ad term')
    plt.plot(log.t[idx_start:idx_end], log.g_term[idx_start:idx_end,i], label='g term')
    # plt.plot(log.t[idx_start:idx_end], log.F_r[idx_start:idx_end,i], label='F_r')
    # plt.plot(log.t[idx_start:idx_end], log.F_r_dot[idx_start:idx_end,i], label='F_r_dot')
    # plt.plot(log.t[idx_start:idx_end], log.F_r_prime[idx_start:idx_end,i], label='F_r_prime')
    plt.legend()
# plt.savefig('plots/plt-Fr-breakdown-' + nametag + '.jpg')
plt.show()
savefig('Fr-breakdown')

In [ ]:
istart = 0

plt.figure()
ax = plt.gca()
plt.plot(log['t'][istart:], np.sqrt(np.sum((log['X'][istart:, 0:3] - log['pd'][istart:,:])**2,1)))
plt.xlabel('t')
plt.ylim(bottom=0)
# plt.yscale('log')
tick_spacing = T / 2
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=tick_spacing))
plt.ylabel('p-error')

plt.show()
savefig('p-error')

In [ ]:
plt.plot(log.t, log.T)
plt.plot(log.t, log.T_r)
plt.legend(('T', 'T_r'))

In [ ]:
plt.figure(figsize=(15,5))
for i in range(4):
    plt.subplot(1,4,i+1)
    plt.plot(log.t, log.q_sp[:,i], label='q_sp,%i'%i)
    plt.plot(log.t, log.X[:,3+i], label='q_%i'%i)
    plt.plot(log.t, log.q_d[:,i], label='q_d,%i'%i)
    # plt.plot(log.t, log.q_d_prime[:,i])
    # plt.legend(('q_%i'%i, 'q_sp,%i'%i, 'q_d,%i'%i, 'q_d_prime,%i'%i))
    plt.legend()
# plt.savefig('plots/plt-q-' + nametag + '.jpg')

plt.show()
savefig('q')

In [ ]:
plt.figure(figsize=(10,10))
istart = -1000
iend = 1000000
# for i in range(3):
i=1
plt.subplot(3,1,1)
plt.plot(log.t[istart:iend], log.X[istart:iend,4+i], label='q_%i'%(i+1))
plt.plot(log.t[istart:iend], log.q_sp[istart:iend,i+1], label='q_sp_%i'%(i+1))
plt.plot(log.t[istart:iend], np.zeros_like(log.t[istart:iend]), 'k:')
plt.legend()

plt.subplot(3,1,2)
plt.plot(log.t[istart:iend], log.w[istart:iend,i], label='w_%i'%i)
plt.plot(log.t[istart:iend], log.w_sp[istart:iend,i], label='w_sp_%i'%i)
plt.plot(log.t[istart:iend], np.zeros_like(log.t[istart:iend]), 'k:')
plt.legend()

plt.subplot(3,1,3)
plt.plot(log.t[istart:iend], log.alpha_sp[istart:iend,i], label='tau_sp,%i'%i)
# plt.plot(log.t, log.alpha_[:,i] * 0.1, label='0.1*alpha_nominal')
plt.plot(log.t[istart:iend], log.tau[istart:iend,i], label='tau_nominal_%i'%i)
plt.plot(log.t[istart:iend], log.tau_s[istart:iend,i] + log.tau[istart:iend,i], label='tau_%i'%i)
plt.plot(log.t[istart:iend], log.tau_s[istart:iend,i], label='tau_s_%i'%i)
# plt.plot(log.t, log.Xdot[:,10+i], label='alpha')
# plt.legend(('q_%i'%i, 'w_%i'%i, 'w_sp,%i'%i, 'alpha,%i'%i, 'Xdot,%i'%i))
plt.legend()
# plt.legend(('w_%i'%i, 'w_sp,%i'%i, 'tau,%i'%i, 'crossterm,%i'%i))
plt.plot(log.t[istart:iend], np.zeros_like(log.t[istart:iend]), 'k:')
# plt.savefig('plots/plt-attitude-' + nametag + '.jpg')
savefig('attitude-' + str(i))

In [ ]:
plt.figure(figsize=(10,10))
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.plot(log.t, log.w[:,i], label='w_%i'%i)
    plt.plot(log.t, log.w_sp[:,i], label='w_sp_%i'%i)
    plt.plot(log.t, np.zeros_like(log.t), 'k:')
    plt.legend()

    plt.legend()
    # plt.legend(('w_%i'%i, 'w_sp,%i'%i, 'tau,%i'%i, 'crossterm,%i'%i))
    plt.plot(log.t, np.zeros_like(log.t), 'k:')
    # plt.savefig('plots/plt-attitude-' + nametag + '.jpg')
    savefig('omega')

plt.show()

In [ ]:
# plt.figure(figsize=(10,10))
# # for i in range(3):
# i=1
# plt.subplot(3,1,1)
# plt.plot(log.t, log.X[:,i], label='p_%i'%(i+1))
# plt.plot(log.t, log.p_d[:,i], label='p_d_%i'%(i+1))
# plt.plot(log.t, np.zeros_like(log.t), 'k:')
# plt.legend()

# plt.subplot(3,1,2)
# plt.plot(log.t, log.X[:,7+i], label='w_%i'%i)
# plt.plot(log.t, log.[:,i], label='w_sp_%i'%i)
# plt.plot(log.t, np.zeros_like(log.t), 'k:')
# plt.legend()

# plt.subplot(3,1,3)
# # plt.plot(log.t, log.alpha_sp[:,i])
# # plt.plot(log.t, log.alpha_[:,i] * 0.1, label='0.1*alpha_nominal')
# plt.plot(log.t, log.tau[:,i], label='tau_nominal_%i'%i)
# plt.plot(log.t, log.tau_s[:,i], label='tau_s_%i'%i)
# plt.plot(log.t, log.tau_s[:,i] + log.tau[:,i], label='tau_%i'%i)
# # plt.plot(log.t, log.Xdot[:,10+i], label='alpha')
# # plt.legend(('q_%i'%i, 'w_%i'%i, 'w_sp,%i'%i, 'alpha,%i'%i, 'Xdot,%i'%i))
# plt.legend()
# # plt.legend(('w_%i'%i, 'w_sp,%i'%i, 'tau,%i'%i, 'crossterm,%i'%i))
# plt.plot(log.t, np.zeros_like(log.t), 'k:')
# plt.savefig('plt-attitude-' + nametag + '.jpg')

In [ ]:
plt.figure(figsize=(15,5))
idx_end = 90000
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.plot(log.t[:idx_end], log.F_r[:idx_end,i])
    plt.plot(log.t[:idx_end], log.F_r_dot[:idx_end,i])
    plt.plot(log.t[:idx_end], log.F_r_prime[:idx_end,i])
    plt.legend(('F_r,%i'%i, 'F_r_dot,%i'%i, 'F_r_prime,%i'%i))
# plt.savefig('plots/plt-Fprime-' + nametag + '.jpg')
plt.show()
savefig('Fprime')

In [ ]:
log.keys()

In [ ]:
# plt.figure(figsize=(15,5))
# for i in range(3):
#     plt.subplot(130 + int(i + 1))

#     # plt.plot(log.t[-1000:], -log.rg[-1000:,0])
#     # plt.plot(log.t[:], log.f[:,i], label='y')
#     plt.plot(log.t[:], log.Fs[:,i], '--', label='Fs')
#     # plt.plot(log.t, log.f_error[:,i], label='f_error')
#     # plt.plot(log.t[:], log.mpddot[:, i], label='mgddot')
#     # plt.plot(log.t, log.g[:,i], label='g')
#     # plt.plot(log.t, log.u[:,i], label='u')
#     # plt.plot(log.t, log.ahat_dot_prediction[:,i], label='ahat_dot_pred')
#     # plt.plot(log.t, log.ahat_dot_tracking[:,i], label='ahat_dot_tracking')
#     # plt.plot(log.t, log.ahat_dot_regularization[:,i], label='ahat_dot_reg')
#     # plt.plot(log.t[:], log.f_hat[:,0,i], label='f_hat')
#     plt.plot(log.t[:], log.f_hat[:,0,i], label='f_hat')
#     plt.legend()
#     rmse = np.sqrt(np.mean((log.Fs[istart:, i] - log.f_hat[istart:, 0, i])**2))
#     print(rmse)
#     # plt.title('rmse = ' + '%.3fN' % rmse + nametag)
#     plt.ylabel(str(i) + ' force')
#     # plt.savefig('force-prediction-x_' + nametag + '.jpg')

# # plt.savefig('plots/force-prediction-x_' + nametag + '.jpg')
# savefig('ahat-update')

In [ ]:
idx_end = 100000
plt.plot(log.t[:idx_end], log.s_oncone[:idx_end], label='s_cone')
# plt.plot(log.t[:idx_end], log.s_onsphere[:idx_end], label='s_sphere')
plt.plot(log.t[:idx_end], np.ones_like(log.t[:idx_end]), label='1')
plt.legend()
plt.show()
# plt.savefig('plots/plt-projection-' + nametag + '.jpg')

In [ ]:
plt.plot(log.t, log.F_r[:,2])
plt.plot(log.t, log.F_r_no_adaptive[:,2])
plt.legend(['Fz_r with adaptive term', 'Fz_r w/o adaptive term']) # CHECK: This labels might be wrong....
plt.show()

In [ ]:
i = 2
idxtmax = 1500

plt.figure(figsize=(10,10))
# plt.plot(log.t, log.f_hat[:,i])
# plt.plot(log.t, -log.Fs[:,i])
# plt.title('Predicted vs actual x side force')
# plt.savefig('x-sideforce-learned-kernel-ss.png')

# plt.plot(log.t[:idxtmax], -log.Fs[:idxtmax,i]-log.f_hat[:idxtmax,i])
plt.plot(log.t[:idxtmax], log.f_error[:idxtmax,i])
plt.plot(log.t[:idxtmax], log.ahat_dot_prediction[:idxtmax,i])
plt.plot(log.t[:idxtmax], log.ahat_dot_tracking[:idxtmax,i])
plt.plot(log.t[:idxtmax], log.ahat_dot_tracking[:idxtmax,i]+log.ahat_dot_prediction[:idxtmax,i])
plt.legend(['measured error', 'dot_pred',  'dot_tracking', 'dot']) # ''dot_tracking',
plt.title('Predicted vs actual z side force')
# plt.savefig('plots/x-sideforce-learned-kernel-ss.png')
plt.show()

In [ ]:
# plt.plot(log.t, log.Fs[:,0]-log.f_cheat[:,0])
# plt.plot(log.t, log.Fs[:,0])
# plt.plot(log.t, log.f_cheat[:,0])
# plt.title('Predicted vs actual x side force')
# plt.savefig('plots/x-sideforce-learned-kernel-ss.png')

In [ ]:
lower_lim = 0
upper_lim = -1
plt.figure(figsize=(15,5))
for i, axis_label in enumerate(('x', 'y', 'z')):
    plt.subplot(1,3,i+1)
    plt.title('axis = %s' % axis_label)
    plt.xlabel('t')
    plt.plot(log.t[lower_lim:upper_lim], log.Fs[lower_lim:upper_lim,i], 'k')
    plt.plot(log.t[lower_lim:upper_lim], log.rg[lower_lim:upper_lim,i], 'r')
    plt.plot(log.t[lower_lim:upper_lim], log.f_hat[lower_lim:upper_lim,i], 'b')
    plt.plot(log.t[lower_lim:upper_lim], log.f[lower_lim:upper_lim,i], 'g')
    plt.legend(('Fs', 'rg', 'f_hat', 'f',))
    plt.plot(log.t[lower_lim:upper_lim], np.cumsum(log.Fs[lower_lim:upper_lim,i])/np.array(range(len(log.Fs[lower_lim:upper_lim,i]))), 'k:')
    plt.plot(log.t[lower_lim:upper_lim], np.cumsum(log.rg[lower_lim:upper_lim,i])/np.array(range(len(log.rg[lower_lim:upper_lim,i]))), 'r:')
    plt.plot(log.t[lower_lim:upper_lim], np.cumsum(log.f_hat[lower_lim:upper_lim,i])/np.array(range(len(log.f_hat[lower_lim:upper_lim,i]))), 'b:')
    plt.plot(log.t[lower_lim:upper_lim], np.cumsum(log.f[lower_lim:upper_lim,i])/np.array(range(len(log.f[lower_lim:upper_lim,i]))), 'g:')

# plt.savefig('plots/fbreakdown-' + nametag + '.png')
savefig('f-breakdown')
plt.show()